In [1]:
credentials_json_path = r"C:\Users\theos\Downloads\google-python-exercises\google-python-exercises\Tracingtest\credentials.json"

In [2]:
!pip install betfairlightweight pandas numpy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

# Change this certs path to wherever you're storing your certificates

# Your credentials.json file should look like this:

# {
#     "username" : "johnsmith123",
#     "password" : "guest",
#     "app_key" : "****************"
# }



with open(credentials_json_path) as f:
    cred = json.load(f)
    my_username = cred['username']
    my_password = cred['password']
    my_app_key = cred['app_key']

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key)



# if you're having issues with certs, you can login this way without using certificates (at your own risk)

# trading = betfairlightweight.APIClient(username=my_username,
#                                        password=my_password,
#                                        app_key=my_app_key
#                                        )

trading.login_interactive()

<LoginResource>

In [4]:
# Grab all event type ids. This will return a list which we will iterate over to print out the Id and the name of the sport
event_types = trading.betting.list_event_types()

sport_ids = pd.DataFrame(
    {
        'Sport': [event_type_object.event_type.name for event_type_object in event_types],
        'ID': [event_type_object.event_type.id for event_type_object in event_types]
    }
).set_index('Sport').sort_index()

sport_ids

,ID
Sport,
American Football,6423
Australian Rules,61420
Baseball,7511
Basketball,7522
Boxing,6
Cricket,4
Cycling,11
Darts,3503
Esports,27454571


In [5]:
event_types = trading.betting.list_event_types()

# Prepare data for DataFrame
data = []
for event_type_object in event_types:
    data.append({
        'Sport': event_type_object.event_type.name,
        'ID': event_type_object.event_type.id,
        'Market Count': event_type_object.market_count
    })

sport_data = pd.DataFrame(data)

# Find the sport with the highest market count
sport_with_most_data = sport_data.loc[sport_data['Market Count'].idxmax()]

print("Sport with the most associated data (based on market count):")
print(sport_with_most_data)

Sport with the most associated data (based on market count):
Sport           Tennis
ID                   2
Market Count      5541
Name: 1, dtype: object


In [6]:
print("📡 Searching for live soccer matches...")

# --- Define the filter for live soccer events ---
# '1' is the EventTypeID for Soccer. Remove this line to search all sports.
live_event_filter = filters.market_filter(
    in_play_only=True,
    event_type_ids=['1'] 
)

# --- Make the API call to list events ---
# Using list_events is efficient for just finding the matches
live_events = trading.betting.list_events(
    filter=live_event_filter
)

live_matches_data = []
if live_events:
    for event_result in live_events:
        event = event_result.event
        live_matches_data.append({
            "Event ID": event.id,
            "Event Name": event.name,
            "Country": event.country_code,
            # The API returns time in UTC. We convert it for readability.
            "Start Time (UTC)": event.open_date.strftime("%Y-%m-%d %H:%M:%S")
        })
    
    # --- Display results in a clean table ---
    live_df = pd.DataFrame(live_matches_data)
    print(f"\n✅ Found {len(live_df)} live matches.")
    display(live_df)
    
else:
    print("\n❌ No live soccer matches found at the moment.")

📡 Searching for live soccer matches...

✅ Found 19 live matches.


,Event ID,Event Name,Country,Start Time (UTC)
0,34524076,SK Artis Brno v MFK Chrudim,CZ,2025-07-21 16:00:00
1,34516077,Unirea Slobozia v Csikszereda,RO,2025-07-21 16:00:00
2,34520842,Agropecuario v Estudiantes de Caseros,AR,2025-07-21 17:00:00
3,34504682,Utsiktens v Helsingborgs,SE,2025-07-21 17:00:00
4,34468907,Lokomotiv Plovdiv v Dobrudzha,BG,2025-07-21 16:00:00
5,34536072,VJS v GrIFK,FI,2025-07-21 15:30:00
6,34536648,Tabasalu JK v Laanemaa Haapsalu,EE,2025-07-21 17:00:00
7,34123714,Brunei DPMM FC v Hougang Utd,SG,2025-03-15 12:30:00
8,34536066,HPS Helsinki v Pallo-Iirot,FI,2025-07-21 15:30:00
9,34513635,Norrkoping v Varnamo,SE,2025-07-21 17:00:00


In [7]:
print("📡 Finding upcoming soccer matches and their odds...")

# --- 1. Find upcoming soccer 'Match Odds' markets ---
soccer_market_filter = filters.market_filter(
    event_type_ids=['1'],  # '1' is the ID for Soccer
    market_type_codes=['MATCH_ODDS']
)

# CORRECTED: Request all necessary data: EVENT, MARKET_START_TIME, and RUNNER_DESCRIPTION
market_catalogues = trading.betting.list_market_catalogue(
    filter=soccer_market_filter,
    market_projection=['EVENT', 'MARKET_START_TIME', 'RUNNER_DESCRIPTION'],
    max_results='15',
    sort='FIRST_TO_START'
)

if not market_catalogues:
    print("❌ No upcoming soccer matches found.")
else:
    # --- 2. Get market books for all found markets efficiently ---
    market_ids = [market.market_id for market in market_catalogues]
    price_filter = filters.price_projection(price_data=['EX_BEST_OFFERS'])
    
    market_books = trading.betting.list_market_book(
        market_ids=market_ids,
        price_projection=price_filter
    )
    
    # Create a dictionary for easy lookup of market books by their ID
    market_book_map = {book.market_id: book for book in market_books}

    # --- 3. Process the data into a readable format ---
    all_matches_data = []
    
    for market_cat in market_catalogues:
        # Skip any malformed catalogue entries
        if not all([market_cat.event, market_cat.market_start_time, market_cat.runners]):
            continue

        market_book = market_book_map.get(market_cat.market_id)
        
        if market_book and market_book.runners:
            match_data = {
                "Event Name": market_cat.event.name,
                "Market Start Time (UTC)": market_cat.market_start_time.strftime("%Y-%m-%d %H:%M")
            }
            
            # This list is now guaranteed to have data
            runner_names = [runner.runner_name for runner in market_cat.runners]
            
            for runner_book in market_book.runners:
                runner_name = next((r.runner_name for r in market_cat.runners if r.selection_id == runner_book.selection_id), "Unknown")
                
                try:
                    best_back_price = runner_book.ex.available_to_back[0].price
                except IndexError:
                    best_back_price = "N/A"
                
                # Check list has enough items before indexing
                if len(runner_names) >= 2 and runner_name == runner_names[0]:
                    match_data["Home Win Odds"] = best_back_price
                elif len(runner_names) >= 2 and runner_name == runner_names[1]:
                    match_data["Away Win Odds"] = best_back_price
                elif runner_name == "The Draw":
                    match_data["Draw Odds"] = best_back_price

            all_matches_data.append(match_data)

    # --- 4. Display results in a pandas DataFrame ---
    odds_df = pd.DataFrame(all_matches_data)
    
    if not odds_df.empty:
        # Ensure all columns exist before trying to reorder them
        cols = ["Event Name", "Home Win Odds", "Draw Odds", "Away Win Odds", "Market Start Time (UTC)"]
        odds_df = odds_df.reindex(columns=cols)
    
    print(f"\n✅ Successfully fetched odds for {len(odds_df)} matches.")
    display(odds_df)

📡 Finding upcoming soccer matches and their odds...

✅ Successfully fetched odds for 15 matches.


,Event Name,Home Win Odds,Draw Odds,Away Win Odds,Market Start Time (UTC)
0,Test W v Test F,N/A,N/A,N/A,2024-11-25 16:00
1,test a v test b,N/A,N/A,N/A,2024-11-30 15:00
2,Test C v Test V,N/A,N/A,N/A,2024-11-30 18:00
3,Armenia U19 v Georgia U19,1.01,1.01,1.01,2025-06-10 14:30
4,VJS v GrIFK,4.5,1.48,7.4,2025-07-21 15:30
5,HPS Helsinki v Pallo-Iirot,150.0,34.0,1.02,2025-07-21 15:30
6,Lokomotiv Plovdiv v Dobrudzha,1.22,5.9,27.0,2025-07-21 16:00
7,Nomme Utd v Flora Tallinn II,1.04,24.0,110.0,2025-07-21 16:00
8,SK Artis Brno v MFK Chrudim,1.88,2.34,1.04,2025-07-21 16:00
9,Unirea Slobozia v Csikszereda,1.12,10.0,80.0,2025-07-21 16:00


In [8]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import os
from datetime import datetime, date, time # Ensure this import is included and run

# --- Authentication (assuming 'trading' object is already logged in) ---
# If not, you would include your login logic here.
# trading = betfairlightweight.APIClient(...)
# trading.login()

print("📡 Searching for today's tennis matches...")

# --- 1. Define the time range for "today" ---
today = date.today()
today_start = datetime.combine(today, time.min) # Midnight at the start of today
today_end = datetime.combine(today, time.max)   # 23:59:59 at the end of today

time_range = filters.time_range(
    from_=today_start.isoformat(),
    to=today_end.isoformat()
)

# --- 2. Create the filter for Tennis (ID '2') within the time range ---
tennis_filter = filters.market_filter(
    event_type_ids=['2'], # '2' is the EventTypeID for Tennis
    market_start_time=time_range
)

# --- 3. Call the API to get the events ---
events = trading.betting.list_events(
    filter=tennis_filter
)

# --- 4. Process and display the results ---
if events:
    todays_matches = [
        {
            "Event Name": event.event.name,
            "Country": event.event.country_code,
            "Start Time (UTC)": event.event.open_date.strftime("%H:%M:%S"),
            "Event ID": event.event.id
        }
        for event in events
    ]
    
    matches_df = pd.DataFrame(todays_matches)
    print(f"\n✅ Found {len(matches_df)} tennis matches for today.")
    display(matches_df)
    
else:
    print("\n❌ No tennis matches found for today.")

📡 Searching for today's tennis matches...

✅ Found 170 tennis matches for today.


,Event Name,Country,Start Time (UTC),Event ID
0,Set 02,None,16:40:00,34538767
1,Set 01,None,16:40:00,34538766
2,Ti Droguet v Garin,None,16:40:00,34538764
3,Set 03,None,16:05:00,34538763
4,Set 02,None,16:05:00,34538762
...,...,...,...,...
165,Set 02,None,18:40:00,34537200
166,Set 01,US,22:00:00,34535920
167,Set 01,US,17:35:00,34539249
168,Set 03,None,18:40:00,34537201


In [9]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import os

# --- Authentication ---
# Assumes 'trading' object is already created and logged in.
# trading = betfairlightweight.APIClient(...)
# trading.login()

print("📡 Analyzing all live tennis markets (Match Odds & Set Betting)...")

# --- 1. Find all live tennis markets ---
live_tennis_filter = filters.market_filter(
    event_type_ids=['2'],
    market_type_codes=['MATCH_ODDS', 'SET_BETTING'],
    in_play_only=True
)

market_catalogues = trading.betting.list_market_catalogue(
    filter=live_tennis_filter,
    market_projection=['RUNNER_DESCRIPTION', 'EVENT', 'MARKET_DESCRIPTION'],
    max_results='100'
)

if not market_catalogues:
    print("\n❌ No live tennis matches found at the moment.")
else:
    market_ids = [cat.market_id for cat in market_catalogues]
    price_filter = filters.price_projection(price_data=['EX_BEST_OFFERS'])
    market_books = trading.betting.list_market_book(
        market_ids=market_ids,
        price_projection=price_filter
    )
    
    catalogue_map = {cat.market_id: cat for cat in market_catalogues}
    
    market_data = []
    
    for book in market_books:
        catalogue = catalogue_map.get(book.market_id)
        if not catalogue or not catalogue.event:
            continue

        for runner in book.runners:
            try:
                best_back = runner.ex.available_to_back[0]
                best_lay = runner.ex.available_to_lay[0]
                runner_name = next((r.runner_name for r in catalogue.runners if r.selection_id == runner.selection_id), "Unknown")
                
                # --- Calculate spread as a percentage ---
                spread_percent = (best_lay.price - best_back.price) / best_back.price
                
                market_data.append({
                    "Event": catalogue.event.name,
                    "Market Name": catalogue.market_name,
                    "Selection": runner_name,
                    "Back Price": best_back.price,
                    "Back Size (£)": best_back.size,
                    "Lay Price": best_lay.price,
                    "Lay Size (£)": best_lay.size,
                    "Spread (%)": spread_percent, # Store as a raw decimal
                    "Market Liquidity (£)": catalogue.total_matched
                })

            except (IndexError, TypeError, ZeroDivisionError):
                continue

    if market_data:
        market_df = pd.DataFrame(market_data)
        
        # --- Format the output for readability ---
        market_df['Market Liquidity (£)'] = market_df['Market Liquidity (£)'].map('{:,.0f}'.format)
        market_df['Spread (%)'] = market_df['Spread (%)'].map('{:.2%}'.format) # Format as percentage
        
        print(f"\n✅ Found {len(market_catalogues)} live markets. Displaying market data:")
        display(market_df.set_index(['Event', 'Market Name', 'Selection']))
    else:
        print("\n❌ Could not retrieve valid odds for any live markets.")

📡 Analyzing all live tennis markets (Match Odds & Set Betting)...


APIError: SportsAPING/v1.0/listMarketBook 
Params: {'marketIds': ['1.245829518', '1.245829524', '1.245836770', '1.245836772', '1.245811661', '1.245811663', '1.245833082', '1.245833084', '1.245802406', '1.245802408', '1.245824653', '1.245848533', '1.245848534', '1.245824648', '1.245850540', '1.245850546', '1.245847771', '1.245847772', '1.245801243', '1.245801246', '1.245846129', '1.245846127', '1.245846695', '1.245846693', '1.245849932', '1.245849933', '1.245849547', '1.245849548', '1.245847640', '1.245847644', '1.245812448', '1.245812454', '1.245855581', '1.245855584', '1.245829538', '1.245829544', '1.245846871', '1.245846872', '1.245840221', '1.245840223', '1.245813592', '1.245813598'], 'priceProjection': {'priceData': ['EX_BEST_OFFERS'], 'exBestOffersOverrides': {}, 'virtualise': True, 'rolloverStakes': False}} 
Exception: None 
Error: {'code': -32099, 'message': 'ANGX-0001', 'data': {'APINGException': {'requestUUID': 'ie1-ang03a-prd-07020939-0022fc6aec', 'errorCode': 'TOO_MUCH_DATA'

### Tennis Opportunities Set

In [ ]:
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
import os
import time
from collections import defaultdict
from datetime import datetime, timedelta

# Set pandas display option to show all rows
pd.set_option('display.max_rows', None)

# --- Authentication ---
# Assumes 'trading' object is already created and logged in.

# --- MARKET MAKER'S RULESET ---
MIN_SPREAD_PCT = 0.025
MIN_MARKET_LIQUIDITY = 500
MIN_AVAILABLE_SIZE = 4
MIN_HEDGE_SCORE = 0.20

print("📡 Analyzing markets with professional tradeability rules...")

# --- 1. Find markets in the next week ---
now = datetime.utcnow()
next_week = now + timedelta(days=7)
time_range = filters.time_range(from_=now, to=next_week)

market_filter = filters.market_filter(
    event_type_ids=['2'],
    market_type_codes=['MATCH_ODDS', 'SET_BETTING'],
    market_start_time=time_range
)
# CORRECTED: Added 'MARKET_START_TIME' to the projection
market_catalogues = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=['RUNNER_DESCRIPTION', 'EVENT', 'MARKET_DESCRIPTION', 'MARKET_START_TIME'],
    max_results='200'
)

if not market_catalogues:
    print("\n❌ No tennis matches found scheduled for the next week.")
else:
    # --- 2. Fetch market books in chunks ---
    market_ids = [cat.market_id for cat in market_catalogues]
    market_books = []
    chunk_size = 40
    for i in range(0, len(market_ids), chunk_size):
        chunk = market_ids[i:i + chunk_size]
        price_filter = filters.price_projection(price_data=['EX_BEST_OFFERS'])
        book_chunk = trading.betting.list_market_book(market_ids=chunk, price_projection=price_filter)
        market_books.extend(book_chunk)
        time.sleep(0.3)
    
    book_map = {book.market_id: book for book in market_books}
    event_markets = defaultdict(list)
    for cat in market_catalogues:
        event_markets[cat.event.id].append(cat)

    # --- 3. Process data and apply ruleset ---
    opportunity_data = []
    
    for event_id, catalogues in event_markets.items():
        match_odds_cat = next((c for c in catalogues if c.market_name == "Match Odds"), None)
        set_betting_cat = next((c for c in catalogues if c.market_name == "Set Betting"), None)

        for cat in catalogues:
            book = book_map.get(cat.market_id)
            if not book: continue
            
            if cat.market_name == "Match Odds" and set_betting_cat:
                hedge_market_liquidity = set_betting_cat.total_matched
            elif cat.market_name == "Set Betting" and match_odds_cat:
                hedge_market_liquidity = match_odds_cat.total_matched
            else:
                hedge_market_liquidity = 0

            for runner in book.runners:
                try:
                    best_back = runner.ex.available_to_back[0]
                    best_lay = runner.ex.available_to_lay[0]
                    
                    spread_score = (best_lay.price - best_back.price) / best_back.price
                    size_component = min(min(best_back.size, best_lay.size), 100) / 100
                    market_health_component = min(cat.total_matched, 50000) / 50000
                    execution_score = (size_component + market_health_component) / 2
                    hedge_score = min(hedge_market_liquidity, 50000) / 50000
                    rating = spread_score * execution_score * hedge_score * 1000
                    
                    is_tradeable = all([
                        spread_score > MIN_SPREAD_PCT,
                        cat.total_matched > MIN_MARKET_LIQUIDITY,
                        best_back.size > MIN_AVAILABLE_SIZE and best_lay.size > MIN_AVAILABLE_SIZE,
                        hedge_score > MIN_HEDGE_SCORE
                    ])
                    
                    runner_name = next((r.runner_name for r in cat.runners if r.selection_id == runner.selection_id), "Unknown")
                    
                    opportunity_data.append({
                        "Tradeable": is_tradeable,
                        "Rating": rating,
                        "Spread (%)": spread_score,
                        "Event": cat.event.name,
                        "Market Name": cat.market_name,
                        "Selection": runner_name,
                        "Start Time (UTC)": cat.market_start_time,
                        "Back Price": best_back.price,
                        "Lay Price": best_lay.price,
                        "Market Liquidity (£)": cat.total_matched
                    })

                except (IndexError, TypeError, ZeroDivisionError):
                    continue

    if opportunity_data:
        # --- 4. Create, sort, and format the final DataFrame ---
        opp_df = pd.DataFrame(opportunity_data)
        opp_df = opp_df.sort_values(by=["Tradeable", "Rating"], ascending=[False, False])
        
        # Format for display
        opp_df['Rating'] = opp_df['Rating'].map('{:.2f}'.format)
        opp_df['Spread (%)'] = opp_df['Spread (%)'].map('{:.2%}'.format)
        opp_df['Market Liquidity (£)'] = opp_df['Market Liquidity (£)'].map('{:,.0f}'.format)
        
        # This formatting will now work correctly
        opp_df['Start Time (UTC)'] = pd.to_datetime(opp_df['Start Time (UTC)'])
        opp_df['Start Time (UTC)'] = opp_df['Start Time (UTC)'].dt.strftime('%Y-%m-%d %H:%M')
        
        print(f"\n✅ Analysis complete. Top rated opportunities:")
        display(opp_df.set_index(['Event', 'Market Name', 'Selection']))
    else:
        print("\n❌ No valid market data could be retrieved.")

### Cycling opportunity set


In [ ]:
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
import os
import time
from datetime import datetime, timedelta

# Set pandas display option to show all rows
pd.set_option('display.max_rows', None)

# --- Authentication ---
# Assumes 'trading' object is already created and logged in.

# --- MARKET MAKER'S RULESET (Adjusted for Cycling) ---
MIN_SPREAD_PCT = 0.025
MIN_MARKET_LIQUIDITY = 2000  # Lowered for typically less liquid cycling markets
MIN_AVAILABLE_SIZE = 4

print("📡 Analyzing cycling markets with professional tradeability rules...")

# --- 1. Find markets in the next week ---
now = datetime.utcnow()
next_week = now + timedelta(days=7)
time_range = filters.time_range(from_=now, to=next_week)

# ADAPTED: Searching for Cycling 'WIN' markets
market_filter = filters.market_filter(
    event_type_ids=['3'],
    market_type_codes=['WIN'],
    market_start_time=time_range
)
market_catalogues = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=['RUNNER_DESCRIPTION', 'EVENT', 'MARKET_DESCRIPTION', 'MARKET_START_TIME'],
    max_results='200'
)

if not market_catalogues:
    print("\n❌ No cycling markets found scheduled for the next week.")
else:
    # --- 2. Fetch market books in chunks ---
    market_ids = [cat.market_id for cat in market_catalogues]
    market_books = []
    chunk_size = 40
    for i in range(0, len(market_ids), chunk_size):
        chunk = market_ids[i:i + chunk_size]
        price_filter = filters.price_projection(price_data=['EX_BEST_OFFERS'])
        book_chunk = trading.betting.list_market_book(market_ids=chunk, price_projection=price_filter)
        market_books.extend(book_chunk)
        time.sleep(0.3)
    
    book_map = {book.market_id: book for book in market_books}

    # --- 3. Process data and apply ruleset (simplified logic) ---
    opportunity_data = []
    
    for cat in market_catalogues:
        book = book_map.get(cat.market_id)
        if not book: continue

        for runner in book.runners:
            try:
                best_back = runner.ex.available_to_back[0]
                best_lay = runner.ex.available_to_lay[0]
                
                # SIMPLIFIED RATING: No hedge score
                spread_score = (best_lay.price - best_back.price) / best_back.price
                size_component = min(min(best_back.size, best_lay.size), 100) / 100
                market_health_component = min(cat.total_matched, 50000) / 50000
                execution_score = (size_component + market_health_component) / 2
                rating = spread_score * execution_score * 1000
                
                # SIMPLIFIED RULES: No hedge rule
                is_tradeable = all([
                    spread_score > MIN_SPREAD_PCT,
                    cat.total_matched > MIN_MARKET_LIQUIDITY,
                    best_back.size > MIN_AVAILABLE_SIZE and best_lay.size > MIN_AVAILABLE_SIZE
                ])
                
                runner_name = next((r.runner_name for r in cat.runners if r.selection_id == runner.selection_id), "Unknown")
                
                opportunity_data.append({
                    "Tradeable": is_tradeable,
                    "Rating": rating,
                    "Spread (%)": spread_score,
                    "Event": cat.event.name,
                    "Selection": runner_name,
                    "Start Time (UTC)": cat.market_start_time,
                    "Back Price": best_back.price,
                    "Lay Price": best_lay.price,
                    "Market Liquidity (£)": cat.total_matched
                })

            except (IndexError, TypeError, ZeroDivisionError):
                continue

    if opportunity_data:
        # --- 4. Create, sort, and format the final DataFrame ---
        opp_df = pd.DataFrame(opportunity_data)
        opp_df = opp_df.sort_values(by=["Tradeable", "Rating"], ascending=[False, False])
        
        # Format for display
        opp_df['Rating'] = opp_df['Rating'].map('{:.2f}'.format)
        opp_df['Spread (%)'] = opp_df['Spread (%)'].map('{:.2%}'.format)
        opp_df['Market Liquidity (£)'] = opp_df['Market Liquidity (£)'].map('{:,.0f}'.format)
        opp_df['Start Time (UTC)'] = pd.to_datetime(opp_df['Start Time (UTC)']).dt.strftime('%Y-%m-%d %H:%M')
        
        print(f"\n✅ Analysis complete. Top rated opportunities:")
        display(opp_df.set_index(['Event', 'Selection']))
    else:
        print("\n❌ No valid market data could be retrieved.")